In [ ]:
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Set up credentials and client
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
client_secret_file = os.getenv("GOOGLE_CLIENT_SECRET_FILE", "clientsecret.json")
sheet_name = os.getenv("GOOGLE_SHEET_NAME", "YOUR_GOOGLE_SHEET_NAME")

creds = ServiceAccountCredentials.from_json_keyfile_name(client_secret_file, scope)
client = gspread.authorize(creds)

# Open your Google Sheet
sheet = client.open(sheet_name).sheet1

# Get all records
data = sheet.get_all_records()
df = pd.DataFrame(data)
df.head()

,Marca temporal,How often do you play video games?,What platform do you play on most?,What game genres do you enjoy the most? (Select up to 2),"When playing a game, what matters most to you?",Do you generally prefer games with:,How important is story/narrative in your enjoyment of a game?,How important are romance options (player-NPC relationships) to your enjoyment of a game when available ?,"When there are romance options, you usually:",Do you prefer games where romance options include:,"When given a choice, what gender do you usually choose for your player character?",How do you describe your gender identity?,How important is that romance options align with your sexual orientation?,How would you describe your sexual orientation?,Have you ever felt more interested in a game because it offered inclusive character or romance options?
0,3/01/2026 11:27:09,Occasionally,PC,RPGs / narrative-driven games,Story and characters,Complex narratives and detailed worlds,5,3,Actively engage with them,No strong preference,Woman,Woman,3,Bisexual,No
1,3/01/2026 11:52:04,Several times a week,Console,RPGs / narrative-driven games,Story and characters,Complex narratives and detailed worlds,4,4,Actively engage with them,Multiple genders and sexual orientations,Woman,Woman,4,Gay / Lesbian,Yes
2,3/01/2026 12:53:35,Several times a week,PC,I can't choose two but it's Action/Shooter and...,Story and characters,Complex narratives and detailed worlds,5,3,Actively engage with them,Limited options (e.g. fixed heterosexual roman...,No preference / depends on the game,Woman,5,Pansexual,No
3,3/01/2026 13:15:28,Several times a week,Console,RPGs / narrative-driven games,Both equally,Complex narratives and detailed worlds,4,4,Actively engage with them,Limited options (e.g. fixed heterosexual roman...,No preference / depends on the game,Non-binary,2,Asexual,Yes
4,3/01/2026 15:23:55,Occasionally,Other,RPGs / narrative-driven games,Story and characters,Complex narratives and detailed worlds,5,4,Actively engage with them,Multiple genders and sexual orientations,Non-binary / Other,Non-binary,4,Bisexual,Yes


In [ ]:
import os
import mysql.connector

# Connect to MySQL using environment variables
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST", "localhost"),
    user=os.getenv("DB_USER", "root"),
    password=os.getenv("DB_PASSWORD", ""),
    database=os.getenv("DB_NAME", "survey_db"),
    use_pure=True
)

cursor = conn.cursor()

# Example: Insert data into MySQL (replace 'df' with your DataFrame)
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO survey_responses (
            timestamp, play_frequency, platform, genres, matters_most, preference,
            story_importance, romance_importance, romance_engagement, romance_preference,
            player_gender, identity, orientation_importance, orientation, inclusive_interest
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, tuple(row))
conn.commit()
cursor.close()
conn.close()

In [ ]:
# --- Custom Survey Dashboard ---
import os
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import mysql.connector

# Explicit dark palette (no near-white colors)
color_discrete = [
    "#0B132B", "#1C2541", "#3A506B", "#5BC0BE", "#6FFFE9",
    "#5E548E", "#9F86C0", "#BE95C4", "#E0B1CB", "#7B2CBF",
    "#C77DFF", "#4895EF", "#4361EE", "#560BAD"
 ]

def fetch_data():
    conn = mysql.connector.connect(
        host=os.getenv("DB_HOST", "localhost"),
        user=os.getenv("DB_USER", "root"),
        password=os.getenv("DB_PASSWORD", ""),
        database=os.getenv("DB_NAME", "survey_db"),
        use_pure=True
    )
    df = pd.read_sql("SELECT * FROM survey_responses", conn)
    conn.close()
    return df

def apply_figure_style(fig, title_text):
    fig.update_layout(
        title={
            "text": f"<b>{title_text}</b>",
            "font": {"family": "Arial", "size": 12, "color": "black"},
            "x": 0.01,
            "xanchor": "left"
        },
        font={"family": "Arial", "size": 9, "color": "black"},
        legend={"font": {"family": "Arial", "size": 9, "color": "black"}},
        plot_bgcolor="white",
        paper_bgcolor="white",
        margin={"l": 40, "r": 20, "t": 56, "b": 40},
    )
    fig.update_xaxes(showgrid=False, tickfont={"family": "Arial", "size": 9, "color": "black"})
    fig.update_yaxes(showgrid=True, gridcolor="#E6E6E6", tickfont={"family": "Arial", "size": 9, "color": "black"})
    return fig

def count_bar(df, col, title, horizontal=False):
    counts = df[col].fillna("Missing").astype(str).value_counts().reset_index()
    counts.columns = ["value", "count"]

    if horizontal:
        fig = px.bar(
            counts.sort_values("count", ascending=True),
            x="count",
            y="value",
            orientation="h",
            color="value",
            color_discrete_sequence=color_discrete,
        )
    else:
        fig = px.bar(
            counts.sort_values("count", ascending=False),
            x="value",
            y="count",
            color="value",
            color_discrete_sequence=color_discrete,
        )

    return apply_figure_style(fig, title)

app = dash.Dash(__name__)

app.layout = html.Div(
    style={"backgroundColor": "white", "padding": "16px", "width": "100%", "minHeight": "100vh"},
    children=[
        html.H1(
            "Survey Dashboard",
            style={
                "color": "black",
                "fontFamily": "Arial",
                "fontWeight": "bold",
                "fontSize": "20px",
                "marginBottom": "12px"
            }
        ),
        dcc.Interval(id="refresh-interval", interval=2 * 1000, n_intervals=0),
        html.Div(id="dashboard-content")
    ]
)

@app.callback(
    dash.dependencies.Output("dashboard-content", "children"),
    [dash.dependencies.Input("refresh-interval", "n_intervals")]
  )
def update_dashboard(_n_intervals):
    df = fetch_data()

    drop_cols = {"timestamp", "marca temporal", "id"}
    df = df.loc[:, [c for c in df.columns if c.lower() not in drop_cols]]

    fig_inclusive = count_bar(
        df,
        "inclusive_interest",
        "Interest Due to Inclusive Options"
    )

    fig_identity = px.histogram(
        df,
        x="identity",
        color="player_gender",
        barmode="group",
        color_discrete_sequence=color_discrete,
    )
    fig_identity = apply_figure_style(fig_identity, "Gender Identity vs. Player Gender Choice")
    fig_identity.update_layout(showlegend=True)

    fig_player_gender = count_bar(df, "player_gender", "Player Gender")

    fig_orientation = px.histogram(
        df,
        x="orientation",
        color="orientation_importance",
        barmode="group",
        color_discrete_sequence=color_discrete,
    )
    fig_orientation = apply_figure_style(fig_orientation, "Sexual Orientation vs. Importance")
    fig_orientation.update_layout(showlegend=True)

    fig_orientation_importance = count_bar(
        df,
        "orientation_importance",
        "Orientation Importance"
    )

    def optimal_graph(col):
        title = col.replace("_", " ").title()
        unique_vals = df[col].nunique(dropna=False)

        if pd.api.types.is_numeric_dtype(df[col]) and unique_vals > 10:
            fig = px.histogram(
                df,
                x=col,
                color_discrete_sequence=color_discrete,
            )
            fig.update_traces(marker_color=color_discrete[2])
            return apply_figure_style(fig, title)

        if unique_vals <= 5:
            return count_bar(df, col, title, horizontal=True)

        return count_bar(df, col, title, horizontal=False)

    remaining_cols = [
        c for c in df.columns
        if c not in [
            "inclusive_interest",
            "identity",
            "player_gender",
            "orientation",
            "orientation_importance",
        ]
    ]

    priority_cards = html.Div(
        [
            html.Div([dcc.Graph(figure=fig_inclusive, style={"height": "420px"})], style={"minWidth": "360px"}),
            html.Div([dcc.Graph(figure=fig_identity, style={"height": "420px"})], style={"minWidth": "360px"}),
            html.Div([dcc.Graph(figure=fig_player_gender, style={"height": "420px"})], style={"minWidth": "360px"}),
            html.Div([dcc.Graph(figure=fig_orientation, style={"height": "420px"})], style={"minWidth": "360px"}),
            html.Div([dcc.Graph(figure=fig_orientation_importance, style={"height": "420px"})], style={"minWidth": "360px"}),
        ],
        style={
            "display": "grid",
            "gridTemplateColumns": "repeat(auto-fit, minmax(420px, 1fr))",
            "gap": "16px",
            "alignItems": "stretch"
        }
    )

    other_graphs = html.Div(
        [
            html.Div(
                [dcc.Graph(figure=optimal_graph(col), style={"height": "360px"})],
                style={"minWidth": "360px"}
            ) for col in remaining_cols
        ],
        style={
            "display": "grid",
            "gridTemplateColumns": "repeat(auto-fit, minmax(420px, 1fr))",
            "gap": "16px",
            "marginTop": "16px",
            "alignItems": "stretch"
        }
    )

    return html.Div([priority_cards, other_graphs])

if __name__ == "__main__":
    app.run_server(debug=True)

C:\Users\julia\AppData\Local\Temp\ipykernel_12324\3794824872.py:23: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

